In [1]:
import os
import sys
import pandas as pd

import logging

logging.getLogger("snowflake").setLevel(logging.WARNING)
logging.getLogger("snowflake.connector").setLevel(logging.WARNING)
logging.getLogger("snowflake.snowpark").setLevel(logging.WARNING)


%pwd
os.chdir("../")

# Add the absolute path to src/ so Python can find automatch
src_path = os.path.abspath("src")
if src_path not in sys.path:
    sys.path.append(src_path)
    
%pwd


'c:\\Users\\fiscarelli\\Desktop\\Progetti\\Manpower IT\\Auto-Match\\Candidates-to-Jobs-Auto-Match-Cortex-AI'

In [2]:
%pwd


'c:\\Users\\fiscarelli\\Desktop\\Progetti\\Manpower IT\\Auto-Match\\Candidates-to-Jobs-Auto-Match-Cortex-AI'

In [3]:
from autoMatch.utils.snowflake_utils import get_snowpark_session
session = get_snowpark_session()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://login.microsoftonline.com/e2ba81b8-03fe-407c-96a1-f4bc0f512e7d/saml2?SAMLRequest=nZJbb%2BIwEIX%2FSuR9TuIYuoAFVEChi9QLKmml7ZtJJmDVsVOP09D%2B%2BjpcpO5D%2B7Bvln1mvuM5M7zclyp4A4vS6BFJIkoC0JnJpd6OyGO6CPskQCd0LpTRMCLvgORyPERRqopParfTD%2FBaA7rAN9LI24cRqa3mRqBErkUJyF3G15PbG84iygUiWOdx5FSSo%2FSsnXMVj%2BOmaaKmExm7jRmlNKaD2KtayS%2FyBVH9zKiscSYz6lyy93%2F6BpHEtNsivMITVqfCqdTHEfxE2RxFyP%2Bk6Spc3a9TEkzOv5sZjXUJdg32TWbw%2BHBzNIDewXQ96FLWjxo%2FtxBqayqIxEdtIUJtmkKJF8hMWdXOd4%2F8KS4gj5XZSj%2Bz5dWIVC8yVx%2BL%2FfU9lrfL17mbd%2F66efp8lz5Ni0JukJYpXO96YjZZsO48I8HTOWHWJrxErGGp21ydv6LsIkxoSHspHXBGeacTsR57JsGV9ye1cIfKs%2FmDj6iUmTVoCme0khoOLoFtRD%2FZ9EPaKSDs0l4WDn6LJCy6m4wWFwmDXh636TFy3CB%2BMGLH%2FzuXYfy1y2kp73xOy6uVUTJ7DxbGlsJ9H2MSJYcbmYfFQcqhFFJN8twCoo9TKdPMLA

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: str
    database: str
    schema: str
    input_table: str
    input_table_cleaned: str
    output_table: str


In [5]:
from autoMatch.constants import *
from autoMatch.utils.common import read_yaml, create_directories
from autoMatch import logger

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_ingestion_config = DataTransformationConfig(
            root_dir=config.root_dir,
            database=config.database,
            schema=config.schema,
            input_table=config.input_table,
            input_table_cleaned=config.input_table_cleaned,
            output_table = config.output_table,
        )

        return data_ingestion_config

In [ ]:
from snowflake.snowpark.functions import col, trim, lower, length, expr
from snowflake.snowpark.types import StringType

from snowflake.snowpark.functions import udf


class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def clean_description(self, session):
        """
        Reads input table
        Cleans description column:
            - removes rows with empty description
            - replaces multiple consecutive whitespaces with a single whitespace (preserves newlines)
            - removes all html tags
            - lowercases all text
        Performs Named Entity Recognition
        Function returns Snowflake dataframe
        """
        database = self.config.database
        schema = self.config.schema
        input_table = self.config.input_table
        input_table_cleaned = self.config.input_table_cleaned

        df = session.table(f"{database}.{schema}.{input_table}").limit(100)

        df = df.filter((col("description").is_not_null()) & (trim(col("description")) != ""))


        def normalize_whitespace(text: str) -> str:
            import re
            return re.sub(r'[ \t]+', ' ', text).strip()

        normalize_whitespace_udf = udf(
            normalize_whitespace, 
            return_type=StringType(), 
            input_types=[StringType()]
            )

        df = df.with_column("description", normalize_whitespace_udf(df["description"]))


        def clean_html(text: str) -> str:
            from bs4 import BeautifulSoup
            if not text:
                return ""
            return BeautifulSoup(text, "html.parser").get_text()

        clean_html_udf = udf(
            clean_html,
            return_type=StringType(),
            input_types=[StringType()],
            packages=["beautifulsoup4"]
            )

        df = df.with_column("description", clean_html_udf(df["description"]))

        df = df.with_column("description", lower(df["description"]))

        df = df.filter((col("description").is_not_null()) & 
                       (trim(col("description")) != "") &
                       (length(trim(col("description"))) > 5) &
                       (~col("description").like("%None%")) &
                       (~col("description").like("%null%"))
                       )

        df = df.with_column("description", col("description").cast("STRING"))
        
        def remove_special_chars(text: str) -> str:
            import re
            if not text:
                return ""
            return re.sub(r'[^a-zA-Z0-9\s]', '', text)
        remove_special_chars_udf = udf(
            remove_special_chars,
            return_type=StringType(),
            input_types=[StringType()]
        )
        df = df.with_column("description", remove_special_chars_udf(df["description"]))

        df.write.save_as_table(
            input_table_cleaned,
            mode="overwrite",
        )
        logger.info(f"Table {input_table} successfully cleaned")

        query = f"""
            SELECT
                *,
                SNOWFLAKE.CORTEX.COMPLETE(
                    'claude-4-sonnet',
                    CONCAT(
                        'Estrai dal seguente testo i campi: age (numero), date_of_birth (YYYY-MM-DD), location (stringa), zip_code (numero), last_job, second_last_job, third_last_job, skills (stringa). ',
                        'Rispondi in formato JSON, senza testo extra, attieniti a questo esempio: {{"age": 30, "date_of_birth": "1993-05-12", "location": "Milano", "zip_code": 20100, "last_job": "Data Engineer", "second_last_job": "Developer", "third_last_job": "Intern", "skills": "Python, SQL"}}. ',
                        'Testo: ', description
                    )
                ) AS parsed_json
            FROM {database}.{schema}.{input_table_cleaned}

            """
         
        import json

        response_format = {
            "età": "Qual è l età del candidato? (numero)",
            "data_di_nascita": "Qual e la data di nascita del candidato? Restituisci solo uno in formato stringa ANNO-MESE-GIORNO",
            "località": "Qual è la località del candidato? Restituisci solo uno in formato stringa",
            "cap": "Qual è il codice postale del candidato? Restituisci solo uno in formato stringa",
            "ultimo_lavoro": "Qual è l ultimo lavoro svolto dal candidato? Restituisci solo uno in formato stringa, il tipo di lavoro, non l azienda o il datore di lavoro",
            "penultimo_lavoro": "Qual è il penultimo lavoro svolto dal candidato? Restituisci solo uno in formato stringa, il tipo di lavoro, non l azienda o il datore di lavoro",
            "terzultimo_lavoro": "Qual è il terzultimo lavoro svolto dal candidato? Restituisci solo uno in formato stringa, il tipo di lavoro, non l azienda o il datore di lavoro",
            "competenze": "Qual è l elenco delle competenze del candidato? (stringa di competenze separate da virgola)"
        }

        response_json = json.dumps(response_format)

        query = f"""
            SELECT 
                *, 
                AI_EXTRACT(
                text => TO_VARCHAR(description),
                responseFormat => PARSE_JSON('{response_json}')
                ) AS ner_json
                FROM {database}.{schema}.{input_table_cleaned}
        """
        
        qq = f"""
                AI_EXTRACT(
                text => description,
                responseFormat => PARSE_JSON('{response_json}')
                )
        """
        qq2 = """ 
                SNOWFLAKE.CORTEX.COMPLETE(
                    'claude-4-sonnet',
                    CONCAT(
                        'Estrai dal seguente testo i campi: age (numero), date_of_birth (YYYY-MM-DD), location (stringa), zip_code (numero), last_job, second_last_job, third_last_job, skills (stringa). ',
                        'Rispondi in formato JSON, senza testo extra, attieniti a questo esempio: {{"age": 30, "date_of_birth": "1993-05-12", "location": "Milano", "zip_code": 20100, "last_job": "Data Engineer", "second_last_job": "Developer", "third_last_job": "Intern", "skills": "Python, SQL"}}. ',
                        'Testo: ', description
                    )
                )
                """

        '''
        df = session.sql(f"SELECT * FROM {database}.{schema}.{input_table_cleaned}")
        descriptions = df.select(col("description").alias("description")).collect()
        rcount = ecount = 0
        for row in descriptions:
            desc = row[0]  # or row["description"] if aliased
            try:
                single_df = df.filter(col("description") == desc)
                single_df = single_df.with_column("ner_json", expr(qq))
                single_df.select("ner_json").show()
                rcount += 1
            except Exception as e:
                ecount += 1
                print(f"Errore {ecount} su: {desc}")
        print(f"Numero totale di errori: {ecount}")
        '''

        df_with_ner = session.sql(query).collect()
        df_with_ner = df_with_ner.filter(df_with_ner["ner_json"].is_not_null())
        df_with_ner = df_with_ner.filter(
            (col("ner_json")["response"].is_not_null()) &
            (trim(col("ner_json")["response"]) != "")
            )

        df_with_ner = df_with_ner.with_columns(
            ["age", "date_of_birth", "location", "zip_code", "last_job", "second_last_job", "third_last_job", "skills"],
            [
                col("ner_json")["response"]["età"].cast("STRING"),
                col("ner_json")["response"]["data_di_nascita"].cast("STRING"),
                col("ner_json")["response"]["località"].cast("STRING"),
                col("ner_json")["response"]["cap"].cast("STRING"), 
                col("ner_json")["response"]["ultimo_lavoro"].cast("STRING"),
                col("ner_json")["response"]["penultimo_lavoro"].cast("STRING"),
                col("ner_json")["response"]["terzultimo_lavoro"].cast("STRING"),
                col("ner_json")["response"]["competenze"].cast("STRING")
            ]
        )

        df_with_ner = df_with_ner.drop("ner_json")

        #print(df_with_ner.show(100))
        logger.info(f"NER on {input_table} table successful")


        return df_with_ner



    def apply_ner_cortexai(self, session):
        """
        Reads input table

        Performs Named Entity Recognition
        Function returns Snowflake dataframe
        """

        from snowflake.snowpark.functions import parse_json


        database = self.config.database
        schema = self.config.schema
        input_table_cleaned = self.config.input_table_cleaned

        query = f"""
            SELECT
                *,
                SNOWFLAKE.CORTEX.COMPLETE(
                    'claude-4-sonnet',
                    CONCAT(
                        'Estrai dal seguente testo i campi: age (numero), date_of_birth (YYYY-MM-DD), location (stringa), zip_code (numero), last_job, second_last_job, third_last_job, skills (stringa). ',
                        'Rispondi in formato JSON, senza testo extra, attieniti a questo esempio: {{"age": 30, "date_of_birth": "1993-05-12", "location": "Milano", "zip_code": 20100, "last_job": "Data Engineer", "second_last_job": "Developer", "third_last_job": "Intern", "skills": "Python, SQL"}}. ',
                        'Testo: ', description
                    )
                ) AS ner_json
            FROM {database}.{schema}.{input_table_cleaned}

            """
         
        import json

        response_format = {
            "età": "Qual è l età del candidato? (numero)",
            "data_di_nascita": "Qual e la data di nascita del candidato? Restituisci solo uno in formato stringa ANNO-MESE-GIORNO",
            "località": "Qual è la località del candidato? Restituisci solo uno in formato stringa",
            "cap": "Qual è il codice postale del candidato? Restituisci solo uno in formato stringa",
            "ultimo_lavoro": "Qual è l ultimo lavoro svolto dal candidato? Restituisci solo uno in formato stringa, il tipo di lavoro, non l azienda o il datore di lavoro",
            "penultimo_lavoro": "Qual è il penultimo lavoro svolto dal candidato? Restituisci solo uno in formato stringa, il tipo di lavoro, non l azienda o il datore di lavoro",
            "terzultimo_lavoro": "Qual è il terzultimo lavoro svolto dal candidato? Restituisci solo uno in formato stringa, il tipo di lavoro, non l azienda o il datore di lavoro",
            "competenze": "Qual è l elenco delle competenze del candidato? (stringa di competenze separate da virgola)"
        }

        response_json = json.dumps(response_format)

        qq = """ 
                SNOWFLAKE.CORTEX.COMPLETE(
                    'claude-4-sonnet',
                    CONCAT(
                        'Estrai dal seguente testo i campi: age (numero), date_of_birth (YYYY-MM-DD), location (stringa), zip_code (numero), last_job, second_last_job, third_last_job, skills (stringa). ',
                        'Rispondi in formato JSON, senza testo extra, attieniti a questo esempio: {{"age": 30, "date_of_birth": "1993-05-12", "location": "Milano", "zip_code": 20100, "last_job": "Data Engineer", "second_last_job": "Developer", "third_last_job": "Intern", "skills": "Python, SQL"}}. ',
                        'Testo: ', description
                    )
                )
                """

        '''
        df = session.sql(f"SELECT * FROM {database}.{schema}.{input_table_cleaned}")
        descriptions = df.select(col("description").alias("description")).collect()
        rcount = ecount = 0
        for row in descriptions:
            desc = row[0]  # or row["description"] if aliased
            try:
                single_df = df.filter(col("description") == desc)
                single_df = single_df.with_column("ner_json", expr(qq))
                single_df.select("ner_json").show()
                rcount += 1
            except Exception as e:
                ecount += 1
                print(f"Errore {ecount} su: {desc}")
        print(f"Numero totale di errori: {ecount}")
        '''

        df = session.sql(query)


        def clean_parsing_text_udf(x: str) -> str:
            if x is None:
                return ''
            x = x.lower().lstrip()

            if x.startswith("```json"):
                x = x[8:].lstrip()

            x = x.replace('\n', ' ').replace('\t', ' ').replace('\\', '').strip()
            x = ' '.join(x.split())

            if x.endswith("```"):
                x = x[:-3].rstrip()
            if x.endswith("'") or x.endswith('"'):
                x = x[:-1].rstrip()

            return x

        clean_udf = udf(
            clean_parsing_text_udf,
            return_type=StringType(),
            input_types=[StringType()]
        )

        df = df.with_column("ner_json", clean_udf(df["ner_json"]))

        df = df.with_column("ner_json", parse_json(col("ner_json")))

        print(df.select(col("ner_json")["age"]).show())

        df = df.filter(df["ner_json"].is_not_null())

        '''
        df = df.filter(
            (col("ner_json")["response"].is_not_null()) &
            (trim(col("ner_json")["response"]) != "")
            )
        '''
        df = df.with_columns(
            ["age", "date_of_birth", "location", "zip_code", "last_job", "second_last_job", "third_last_job", "skills"],
            [
                col("ner_json")["age"].cast("STRING"),
                col("ner_json")["data_of_birth"].cast("STRING"),
                col("ner_json")["location"].cast("STRING"),
                col("ner_json")["zip_code"].cast("STRING"), 
                col("ner_json")["last_job"].cast("STRING"),
                col("ner_json")["second_last_job"].cast("STRING"),
                col("ner_json")["third_last_job"].cast("STRING"),
                col("ner_json")["skills"].cast("STRING")
            ]
        )

        df = df.drop("ner_json")

        print(df.show(100))
        logger.info(f"NER on {input_table_cleaned} table successful")


        return df
    

    def apply_ner_rowwise(self, session):

        from snowflake.snowpark import Row
        from snowflake.snowpark.functions import lit
        from functools import reduce

        database = self.config.database
        schema = self.config.schema
        input_table_cleaned = self.config.input_table_cleaned

        response_format = {
            "età": "Qual è l età del candidato? (numero)",
            "data_di_nascita": "Qual e la data di nascita del candidato? Restituisci solo uno in formato stringa ANNO-MESE-GIORNO",
            "località": "Qual è la località del candidato? Restituisci solo uno in formato stringa",
            "cap": "Qual è il codice postale del candidato? Restituisci solo uno in formato stringa",
            "ultimo_lavoro": "Qual è l ultimo lavoro svolto dal candidato? Restituisci solo uno in formato stringa, il tipo di lavoro, non l azienda o il datore di lavoro",
            "penultimo_lavoro": "Qual è il penultimo lavoro svolto dal candidato? Restituisci solo uno in formato stringa, il tipo di lavoro, non l azienda o il datore di lavoro",
            "terzultimo_lavoro": "Qual è il terzultimo lavoro svolto dal candidato? Restituisci solo uno in formato stringa, il tipo di lavoro, non l azienda o il datore di lavoro",
            "competenze": "Qual è l elenco delle competenze del candidato? (stringa di competenze separate da virgola)"
        }

        import json
        response_json = json.dumps(response_format)

        rows = session.table(f"{database}.{schema}.{input_table_cleaned}").collect()
        results = []
        for row in rows:
            try:
                candidate_id = row["CANDIDATEID"]
                description = row["description"]

                df_single = session.create_dataframe(
                    [[candidate_id, description]],
                    schema=["CANDIDATEID", "description"]
                )

                df_single = df_single.with_column(
                    "ner_json",
                    expr(f"""
                        AI_EXTRACT(
                            text => description,
                            responseFormat => PARSE_JSON('{response_json}')
                        )
                    """)
                )

                results.append(df_single)
            except Exception as e:
                logger.warning(f"Errore su CANDIDATEID={candidate_id}: {e}")

        if not results:
            raise ValueError("Nessuna riga è stata processata correttamente da AI_EXTRACT.")

        final_df = reduce(lambda a, b: a.union(b), results)

        final_df = final_df.with_columns(
            ["age", "date_of_birth", "location", "zip_code", "last_job", "second_last_job", "third_last_job", "skills"],
            [
                col("ner_json")["response"]["età"].cast("STRING"),
                col("ner_json")["response"]["data_di_nascita"].cast("STRING"),
                col("ner_json")["response"]["località"].cast("STRING"),
                col("ner_json")["response"]["cap"].cast("STRING"),
                col("ner_json")["response"]["ultimo_lavoro"].cast("STRING"),
                col("ner_json")["response"]["penultimo_lavoro"].cast("STRING"),
                col("ner_json")["response"]["terzultimo_lavoro"].cast("STRING"),
                col("ner_json")["response"]["competenze"].cast("STRING")
            ]
        )

        logger.info("NER row-wise completato con successo")
        return final_df


    def write_table(self, session, df):
        """
        Writes input table
        Function returns nothing
        """
        output_table = self.config.output_table

        df.write.save_as_table(output_table, mode="overwrite")
        logger.info(f"Table {output_table} successfully written")

  


In [55]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    #df = data_transformation.clean_description(session)
    df = data_transformation.apply_ner_cortexai(session)
    data_transformation.write_table(session, df)

except Exception as e:
    raise e

[2025-10-07 11:51:12,889: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-07 11:51:12,893: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-07 11:51:12,897: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-10-07 11:51:12,901: INFO: common: created directory at: artifacts]
[2025-10-07 11:51:12,903: INFO: common: created directory at: artifacts/data_transformation]
-------------------------
|"""NER_JSON""['AGE']"  |
-------------------------
|36                     |
|null                   |
|22                     |
|37                     |
|null                   |
|18                     |
|54                     |
|24                     |
|22                     |
|23                     |
-------------------------

None
-------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"DATE_ADDED"  |"CANDIDATEID"  |"DESCRIPT

In [56]:
df_a = session.sql("SELECT * FROM MPG_IT_AUTOMATCH_CANDIDATE_FEATURES").to_pandas()
df_b = session.sql("SELECT * FROM AUTOMATCH_FINAL_TABLE").to_pandas()

df_a["CANDIDATEID"] = df_a["CANDIDATEID"].astype(str)
df_b["CANDIDATEID"] = df_b["CANDIDATEID"].astype(str)

df_merged = pd.merge(df_a, df_b, on="CANDIDATEID", suffixes=("_a", "_b"))
df_merged = df_merged[sorted(df_merged.columns)]
df_merged

,AGE_a,AGE_b,CANDIDATEID,DATE_ADDED,DATE_OF_BIRTH_a,DATE_OF_BIRTH_b,DESCRIPTION,LAST_JOB_a,LAST_JOB_b,LOCATION_a,LOCATION_b,SECOND_LAST_JOB_a,SECOND_LAST_JOB_b,SKILLS_a,SKILLS_b,THIRD_LAST_JOB_a,THIRD_LAST_JOB_b,ZIP_CODE_a,ZIP_CODE_b


In [57]:
result = session.sql(f"""SELECT AI_EXTRACT(
  text => 'Antonio Cavalli Software Engineer Location: Milano',
  responseFormat => PARSE_JSON('{{"name": "What is the first name of the employee?", "address": "What is the address of the employee?"}}')
 
);""").collect()
print(result[0][0])



{
  "response": {
    "address": "Milano",
    "name": "Antonio"
  }
}


In [58]:

df = session.range(1).select(
    ai_extract(
        'John Smith lives in San Francisco and works for Snowflake',
        {'name': 'What is the first name of the employee?', 'city': 'What is the address of the employee?'}
    ).alias("extracted")
)
df.show()

NameError: name 'ai_extract' is not defined